Running notebook will generate annotations files for HOI Transformer using 2.5VRD annotation files. 

Resulting annotations will be written to files:

annotation_train_distances.odgt

annotation_train_occlusions.odgt

annotation_valid_distances.odgt

annotation_valid_occlusions.odgt

annotation_test_distances.odgt

annotation_test_occlusions.odgt

In [1]:
import pandas as pd
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import sys
import json
import time
import os

# Load 2.5VRD Annotation Files

In [2]:
within_image_objects_train = pd.read_csv('within_image_objects_train.csv')
within_image_vrd_train = pd.read_csv('within_image_vrd_train.csv')

within_image_objects_validation = pd.read_csv('within_image_objects_validation.csv')
within_image_vrd_validation = pd.read_csv('within_image_vrd_validation.csv')

within_image_objects_test = pd.read_csv('within_image_objects_test.csv')
within_image_vrd_test = pd.read_csv('within_image_vrd_test.csv')

# Load and Process class_descriptions_boxable

In [3]:
class_descriptions_boxable = pd.read_csv('class-descriptions-boxable.csv', header=None)

# Insert a row at the top
class_descriptions_boxable.loc[-1] = ['','']
class_descriptions_boxable.index = class_descriptions_boxable.index + 1
class_descriptions_boxable.sort_index(inplace=True)

class_descriptions_boxable

,0,1
0,,
1,/m/011k07,Tortoise
2,/m/011q46kg,Container
3,/m/012074,Magpie
4,/m/0120dh,Sea turtle
...,...,...
597,/m/0qmmr,Wheelchair
598,/m/0wdt60w,Rugby ball
599,/m/0xfy,Armadillo
600,/m/0xzly,Maracas


# Define Functions

In [4]:
def progressBar(i, max, text):
    """
    Produce a progress bar during training.
    :param i: index of current iteration/epoch.
    :param max: max number of iterations/epochs.
    :param text: Text to print on the right of the progress bar.
    :return: None
    """
    bar_size = 60
    j = i / max
    
    sys.stdout.write('\r')
    sys.stdout.write(' ' * bar_size * 5)
    sys.stdout.write('\r')
    sys.stdout.write(
        f"[{'=' * int(bar_size * j):{bar_size}s}] {int(100 * j)}%  {text}")
    sys.stdout.flush()
    
def entity_to_name(entity):
    return class_descriptions_boxable[class_descriptions_boxable[0] == entity][1].item()


def name_to_entity(name):
    return class_descriptions_boxable[class_descriptions_boxable[1] == name][0].item()

def entity_to_index(entity):
    index = -1
    try:
        index = class_descriptions_boxable[class_descriptions_boxable[0] == entity].index.item()
    except:
        pass
    return index

def name_to_index(name):
    index = -1
    try:
        index = class_descriptions_boxable[class_descriptions_boxable[1] == name].index.item()
    except:
        pass
    return index


distance_id_to_name = {
    -1: 'no majority',
    0: 'not sure',
    1: 'A is closer',
    2: 'B is closer',
    3: 'same distance'
}

distance_name_to_id = {
    'no majority': -1,
    'not sure': 0,
    'A is closer': 1,
    'B is closer': 2,
    'same distance': 3
}

occlusion_id_to_name = {
    -1: 'no majority',
    0: 'no occlusion',
    1: 'A occludes B',
    2: 'B occludes A',
    3: 'mutually occluded'
}

occlusion_name_to_id = {
    'no majority': -1,
    'no occlusion': 0,
    'A occludes B': 1,
    'B occludes A': 2,
    'mutually occluded': 3
}

# EDA

## Training Set

In [5]:
within_image_objects_train

,image_id,object_id,entity,xmin,xmax,ymin,ymax
0,000002b66c9c498e,0,/m/01g317,0.151562,0.310937,0.198437,0.590625
1,000002b66c9c498e,1,/m/01g317,0.025000,0.276563,0.714063,0.948438
2,00003d63a4839019,0,/m/07j7r,0.681250,0.999375,0.000000,0.647280
3,00003d63a4839019,1,/m/07j7r,0.000000,0.699375,0.000000,0.668856
4,0000599864fd15b3,0,/m/03bt1vf,0.199375,0.350625,0.444967,0.999059
...,...,...,...,...,...,...,...
211383,fffe519ee3cddd18,1,/m/04yx4,0.223125,0.863125,0.231667,1.000000
211384,fffe53b2f679f98c,0,/m/0dzct,0.349578,0.664480,0.404375,0.710000
211385,fffe53b2f679f98c,1,/m/04yx4,0.025305,0.813496,0.292500,0.988125
211386,ffff1cb23ed29f2a,0,/m/09j2d,0.649375,0.853750,0.618333,0.999167


In [6]:
within_image_vrd_train

,image_id_1,object_id_1,image_id_2,object_id_2,distance,occlusion,raw_distance,raw_occlusion
0,000002b66c9c498e,0,000002b66c9c498e,1,3,0,3,0
1,00003d63a4839019,0,00003d63a4839019,1,3,3,3,3
2,0000599864fd15b3,0,0000599864fd15b3,1,1,0,1,0
3,000060e3121c7305,0,000060e3121c7305,1,0,0,0,0
4,000096726fd6c6c8,0,000096726fd6c6c8,1,2,2,2,2
...,...,...,...,...,...,...,...,...
105655,fffd4b6d215e5850,0,fffd4b6d215e5850,1,1,0,1,0
105656,fffe28b31f2a70d4,0,fffe28b31f2a70d4,1,0,0,0,0
105657,fffe519ee3cddd18,0,fffe519ee3cddd18,1,0,0,0,0
105658,fffe53b2f679f98c,0,fffe53b2f679f98c,1,0,0,"0,0,0","0,0,0"


### Investigate the number of unique images

In [7]:
print('number of  unique images in within_image_objects_train:    ', within_image_objects_train['image_id'].unique().shape[0])
print('number of  unique images in within_image_vrd_train:        ', within_image_vrd_train['image_id_1'].unique().shape[0])
print('number of  unique images after combining the above two:    ', pd.concat([within_image_objects_train['image_id'], within_image_vrd_train['image_id_1']]).unique().shape[0])

number of  unique images in within_image_objects_train:     105694
number of  unique images in within_image_vrd_train:         105660
number of  unique images after combining the above two:     105694


Since the number of unique images after combining two datasets is the same as the number of images in the within_image_objects_train, within_image_objects_train contains all the images names.

In other words, there are some images that have annotations for objects but no annotation for visual relationships.

We do not want to trian on images which have annotations for objects but no annotation for visual relationships. 

Therefore, we use image names and visual relationships from the vrd annotations, and find their object bounding boxes from the object annotations. 

## Validation Set

In [8]:
within_image_objects_validation

,image_id,object_id,entity,xmin,xmax,ymin,ymax
0,0071f62f5d703904,0,/m/02p5f1q,0.122163,0.345615,0.088189,0.341194
1,0071f62f5d703904,1,/m/02p5f1q,0.167023,0.481887,0.392021,0.773787
2,0071f62f5d703904,2,/m/02p5f1q,0.369314,0.578377,0.123204,0.410093
3,0071f62f5d703904,3,/m/02p5f1q,0.504740,0.854307,0.441718,0.855110
4,0071f62f5d703904,4,/m/02vqfm,0.602288,0.835815,0.455359,0.651093
...,...,...,...,...,...,...,...
4058,ffb9838816c0021a,2,/m/04yx4,0.132984,0.286043,0.110463,0.490701
4059,ffb9838816c0021a,3,/m/083wq,0.569882,0.700175,0.602955,0.946547
4060,ffeba8a4d2029eac,0,/m/04m6gz,0.000000,1.000000,0.256178,0.666241
4061,ffeba8a4d2029eac,1,/m/07j7r,0.000000,1.000000,0.000000,0.602860


In [9]:
within_image_vrd_validation

,image_id_1,object_id_1,image_id_2,object_id_2,distance,occlusion,raw_distance,raw_occlusion
0,0071f62f5d703904,0,0071f62f5d703904,1,2,0,"2,2,2,2,2","0,0,0,0,0"
1,0071f62f5d703904,0,0071f62f5d703904,2,2,0,"2,2,2,2,2","0,0,0,0,0"
2,0071f62f5d703904,0,0071f62f5d703904,3,2,0,"2,2,2,2,2","0,0,0,0,0"
3,0071f62f5d703904,0,0071f62f5d703904,4,2,0,"2,2,2,2,2","0,0,0,0,0"
4,0071f62f5d703904,1,0071f62f5d703904,2,1,0,"1,1,1,2,1","0,0,0,0,0"
...,...,...,...,...,...,...,...,...
6320,ffb9838816c0021a,1,ffb9838816c0021a,2,1,1,"1,1,1,1,1","1,1,1,1,1"
6321,ffb9838816c0021a,2,ffb9838816c0021a,3,2,0,"2,2,2,2,2","0,0,0,0,0"
6322,ffeba8a4d2029eac,0,ffeba8a4d2029eac,1,1,1,"1,1,1,1,1","1,1,1,1,1"
6323,ffeba8a4d2029eac,0,ffeba8a4d2029eac,2,1,1,"1,1,1,1,1","1,1,1,1,0"


### Investigate the number of unique images

In [10]:
print('number of  unique images in within_image_objects_validation:    ', within_image_objects_validation['image_id'].unique().shape[0])
print('number of  unique images in within_image_vrd_validation:        ', within_image_vrd_validation['image_id_1'].unique().shape[0])
print('number of  unique images after combining the above two:         ', pd.concat([within_image_objects_validation['image_id'], within_image_vrd_validation['image_id_1']]).unique().shape[0])

number of  unique images in within_image_objects_validation:     1200
number of  unique images in within_image_vrd_validation:         1196
number of  unique images after combining the above two:          1200


Since the number of unique images after combining two datasets is the same as the number of images in the within_image_objects_validation, within_image_objects_validation contains all the images names.

In other words, there are some images that have annotations for objects but no annotation for visual relationships.

We do not want to validate on images which have annotations for objects but no annotation for visual relationships. 

Therefore, we use image names and visual relationships from the vrd annotations, and find their object bounding boxes from the object annotations. 

## Test Set

In [11]:
within_image_objects_test

,image_id,object_id,entity,xmin,xmax,ymin,ymax
0,0002ab0af02e4a77,0,/m/09j2d,0.407782,0.845727,0.385412,0.999711
1,0002ab0af02e4a77,1,/m/04hgtk,0.534108,0.745821,0.127420,0.462402
2,0003d84e0165d630,0,/m/01g317,0.735415,0.837862,0.014350,0.384812
3,0003d84e0165d630,1,/m/018xm,0.000000,0.198794,0.665719,0.942062
4,000b6e5bfa3e2a34,0,/m/04yx4,0.000000,0.387860,0.174753,0.966402
...,...,...,...,...,...,...,...
13888,ffe04404f471a3e2,5,/m/01_bhs,0.584049,1.000000,0.592052,1.000000
13889,ffe1d00be52c3142,0,/m/07j87,0.036083,0.491698,0.203271,0.787928
13890,ffe1d00be52c3142,1,/m/07j87,0.479360,0.964239,0.300079,0.861355
13891,fffc2ecd02883c6f,0,/m/04hgtk,0.503194,0.828060,0.017568,0.614981


In [12]:
within_image_vrd_test

,image_id_1,object_id_1,image_id_2,object_id_2,distance,occlusion,raw_distance,raw_occlusion
0,0002ab0af02e4a77,0,0002ab0af02e4a77,1,0,0,"0,0,0,0,0","0,0,0,0,0"
1,0003d84e0165d630,0,0003d84e0165d630,1,2,0,"2,2,2,2,2","0,0,0,0,0"
2,000b6e5bfa3e2a34,0,000b6e5bfa3e2a34,1,1,0,"1,0,1,1,1","0,0,1,1,0"
3,000b6e5bfa3e2a34,0,000b6e5bfa3e2a34,2,1,1,"1,1,1,1,1","1,1,1,0,1"
4,000b6e5bfa3e2a34,0,000b6e5bfa3e2a34,3,1,0,"1,1,1,1,1","0,0,0,0,0"
...,...,...,...,...,...,...,...,...
23661,ffe04404f471a3e2,3,ffe04404f471a3e2,4,2,0,"2,2,2,3,2","0,1,0,0,0"
23662,ffe04404f471a3e2,3,ffe04404f471a3e2,5,2,0,"2,2,2,2,2","0,0,0,0,0"
23663,ffe04404f471a3e2,5,ffe04404f471a3e2,4,1,1,"1,1,1,1,1","1,1,1,1,1"
23664,ffe1d00be52c3142,0,ffe1d00be52c3142,1,2,0,"3,2,2,2,3","0,0,0,0,0"


### Investigate the number of unique images

In [13]:
print('number of  unique images in within_image_objects_test:    ', within_image_objects_test['image_id'].unique().shape[0])
print('number of  unique images in within_image_vrd_test:        ', within_image_vrd_test['image_id_1'].unique().shape[0])
print('number of  unique images after combining the above two:   ', pd.concat([within_image_objects_test['image_id'], within_image_vrd_test['image_id_1']]).unique().shape[0])

number of  unique images in within_image_objects_test:     4000
number of  unique images in within_image_vrd_test:         3987
number of  unique images after combining the above two:    4000


Since the number of unique images after combining two datasets is the same as the number of images in the within_image_objects_test, within_image_objects_test contains all the images names.

In other words, there are some images that have annotations for objects but no annotation for visual relationships.

We do not want to test on images which have annotations for objects but no annotation for visual relationships. 

Therefore, we use image names and visual relationships from the vrd annotations, and find their object bounding boxes from the object annotations. 

# Define Functions to Build Annotations and Write

In [14]:
# dataset: 'train', 'valid', or 'test'
def build_annotations(dataset):
    
    # Select vrd DataFrame and objects DataFrame
    if dataset == 'train':
        vrd_df = within_image_vrd_train
        object_df = within_image_objects_train
    elif dataset == 'valid':
        vrd_df = within_image_vrd_validation
        object_df = within_image_objects_validation
        dataset = 'validation'
    elif dataset == 'test':
        vrd_df = within_image_vrd_test
        object_df = within_image_objects_test
        
    
    # Find out unique image ids
    unique_image_ids = vrd_df['image_id_1'].unique()

    all_distances = []
    all_occlusions = []

    skipped_images_ids = []
    skipped_indices = []
    num_skipped_images = 0

    num_images = len(unique_image_ids)

    start_time = time.time()
    time_used = 0

    for i in range(num_images):

        # find file name
        image_id = unique_image_ids[i]
        original_image_id = image_id

        # pad zeros to image_id
        num_leading_zeros = 16 - len(image_id)

        for j in range(num_leading_zeros):
            image_id = '0' + image_id


        file_name = image_id + '.jpg'
        path = 'images/' + dataset + '/' + file_name

        try:
            image = Image.open(path)
        except:
            skipped_images_ids.append(original_image_id)
            skipped_indices.append(i)
            num_skipped_images += 1
            continue

        # find width and height
        width = image.width
        height = image.height

        # find bounding boxes from object_df
        df = object_df[object_df['image_id'] == image_id]

        # Box_array in odgt annotaions should be in xywh format
        box_array = [None] * df.shape[0]
        tag_array = [None] * df.shape[0]

        for j in range(df.shape[0]):
            object_id = df.iloc[j]['object_id']
            xmin = df.iloc[j]['xmin'] * width
            ymin = df.iloc[j]['ymin'] * height
            xmax = df.iloc[j]['xmax'] * width
            ymax = df.iloc[j]['ymax'] * height

            entity = df.iloc[j]['entity']
            tag = entity_to_name(entity)

            box_array[j] = [xmin, ymin, xmax - xmin, ymax - ymin]
            tag_array[j] = tag

        # Find occlusion and distance relationships from vrd_df
        df = vrd_df[vrd_df['image_id_1'] == image_id]

        pair_array = [None] * (df.shape[0])
        distance_array = [None] * (df.shape[0])
        occlusion_array = [None] * (df.shape[0])

        for j in range(df.shape[0]):
            object_id_1 = df.iloc[j]['object_id_1']
            object_id_2 = df.iloc[j]['object_id_2']
            distance = df.iloc[j]['distance']
            occlusion = df.iloc[j]['occlusion']

            pair_array[j] = [object_id_1, object_id_2]
            distance_array[j] = distance
            occlusion_array[j] = occlusion

        # Build distance and occlusion
        distance = []
        occlusion = []
        gtboxes = []

        # Boxes 
        for j in range(len(box_array)):
            gtboxes.append(dict(box=box_array[j], tag=tag_array[j]))

        # Distances and Occlusions
        for j in range(len(pair_array)):        
            subject_id = pair_array[j][0]
            object_id = pair_array[j][1]

            # distance
            category_id = distance_array[j]
            interaction = distance_id_to_name[category_id]

            dist_pp = dict(subject_id=str(subject_id),
                           object_id=str(object_id),
                           category_id=str(category_id), 
                           interaction=interaction
                           )
            distance.append(dist_pp)



            # occlusion
            category_id = occlusion_array[j]
            interaction = occlusion_id_to_name[category_id]

            occl_pp = dict(subject_id=str(subject_id),
                           object_id=str(object_id),
                           category_id=str(category_id), 
                           interaction=interaction
                           )
            occlusion.append(occl_pp)



        all_distances.append(dict(file_name=file_name, width=width, height=height, gtboxes=gtboxes, hoi=distance, ID=''))
        all_occlusions.append(dict(file_name=file_name, width=width, height=height, gtboxes=gtboxes, hoi=occlusion, ID=''))


        if (i + 1) % 10 == 0 or i + 1== num_images:
            fixed_status_string = 'Generating Annotations'
            dynamic_status_string = str(i + 1) + '/' + str(num_images)

            time_used = time.time() - start_time
            speed = (i + 1) / time_used
            time_remaining = (num_images - (i + 1)) / speed
            remaining_hours = time_remaining / 3600
            remaining_minutes = (time_remaining % 3600) / 60
            remaining_seconds = time_remaining % 60
            time_string = str(int(remaining_hours)) + ' h ' + str(int(remaining_minutes)) +' min ' + str(int(remaining_seconds)) + ' s'


            progressBar(i + 1, num_images, '  ' + fixed_status_string + '  ' + dynamic_status_string + '  ' + '|  Skipped Images: ' + str(num_skipped_images) + '  |  ' + 'Time Remining: ' + time_string)
            
    return all_distances, all_occlusions, skipped_images_ids, skipped_indices, num_skipped_images

In [5]:
# dataset: 'train', 'valid', or 'test'
def build_combined_annotations(dataset):

    # Select vrd DataFrame and objects DataFrame
    if dataset == 'train':
        vrd_df = within_image_vrd_train
        object_df = within_image_objects_train
    elif dataset == 'valid':
        vrd_df = within_image_vrd_validation
        object_df = within_image_objects_validation
        dataset = 'validation'
    elif dataset == 'test':
        vrd_df = within_image_vrd_test
        object_df = within_image_objects_test


    # Find out unique image ids
    unique_image_ids = vrd_df['image_id_1'].unique()

    all_distances = []
    all_occlusions = []

    all_combined = []

    skipped_images_ids = []
    skipped_indices = []
    num_skipped_images = 0

    num_images = len(unique_image_ids)

    start_time = time.time()
    time_used = 0

    for i in range(num_images):

        # find file name
        image_id = unique_image_ids[i]
        original_image_id = image_id

        # pad zeros to image_id
        num_leading_zeros = 16 - len(image_id)

        for j in range(num_leading_zeros):
            image_id = '0' + image_id


        file_name = image_id + '.jpg'
        path = 'images/' + dataset + '/' + file_name

        try:
            image = Image.open(path)
        except:
            skipped_images_ids.append(original_image_id)
            skipped_indices.append(i)
            num_skipped_images += 1
            continue

        # find width and height
        width = image.width
        height = image.height

        # find bounding boxes from object_df
        df = object_df[object_df['image_id'] == image_id]

        # Box_array in odgt annotaions should be in xywh format
        box_array = [None] * df.shape[0]
        tag_array = [None] * df.shape[0]

        for j in range(df.shape[0]):
            object_id = df.iloc[j]['object_id']
            xmin = df.iloc[j]['xmin'] * width
            ymin = df.iloc[j]['ymin'] * height
            xmax = df.iloc[j]['xmax'] * width
            ymax = df.iloc[j]['ymax'] * height

            entity = df.iloc[j]['entity']
            tag = entity_to_name(entity)

            box_array[j] = [xmin, ymin, xmax - xmin, ymax - ymin]
            tag_array[j] = tag

        # Find occlusion and distance relationships from vrd_df
        df = vrd_df[vrd_df['image_id_1'] == image_id]

        pair_array = [None] * (df.shape[0])
        distance_array = [None] * (df.shape[0])
        occlusion_array = [None] * (df.shape[0])

        for j in range(df.shape[0]):
            object_id_1 = df.iloc[j]['object_id_1']
            object_id_2 = df.iloc[j]['object_id_2']
            distance = df.iloc[j]['distance']
            occlusion = df.iloc[j]['occlusion']

            pair_array[j] = [object_id_1, object_id_2]
            distance_array[j] = distance
            occlusion_array[j] = occlusion

        # Build distance and occlusion
        distance = []
        occlusion = []
        gtboxes = []

        # Boxes
        for j in range(len(box_array)):
            gtboxes.append(dict(box=box_array[j], tag=tag_array[j]))

        # Distances and Occlusions
        for j in range(len(pair_array)):
            subject_id = pair_array[j][0]
            object_id = pair_array[j][1]

            # distance
            category_id = distance_array[j]
            interaction = distance_id_to_name[category_id]

            dist_pp = dict(subject_id=str(subject_id),
                           object_id=str(object_id),
                           category_id=str(category_id),
                           interaction=interaction
                           )
            distance.append(dist_pp)



            # occlusion
            category_id = occlusion_array[j]
            interaction = occlusion_id_to_name[category_id]

            occl_pp = dict(subject_id=str(subject_id),
                           object_id=str(object_id),
                           category_id=str(category_id),
                           interaction=interaction
                           )
            occlusion.append(occl_pp)


        all_combined.append(dict(file_name=file_name, width=width, height=height, gtboxes=gtboxes, hoi=distance, occlusion=occlusion, ID=''))


        if (i + 1) % 10 == 0 or i + 1== num_images:
            fixed_status_string = 'Generating Annotations'
            dynamic_status_string = str(i + 1) + '/' + str(num_images)

            time_used = time.time() - start_time
            speed = (i + 1) / time_used
            time_remaining = (num_images - (i + 1)) / speed
            remaining_hours = time_remaining / 3600
            remaining_minutes = (time_remaining % 3600) / 60
            remaining_seconds = time_remaining % 60
            time_string = str(int(remaining_hours)) + ' h ' + str(int(remaining_minutes)) +' min ' + str(int(remaining_seconds)) + ' s'

            progressBar(i + 1, num_images, '  ' + fixed_status_string + '  ' + dynamic_status_string + '  ' + '|  Skipped Images: ' + str(num_skipped_images) + '  |  ' + 'Time Remining: ' + time_string)

    return all_combined, skipped_images_ids, skipped_indices, num_skipped_images


In [12]:
all_combined, skipped_images_ids, skipped_indices, num_skipped_images = build_combined_annotations('train')

In [13]:
all_combined

[]

In [15]:
# file_name: file name of distance annotation file
def write_distances(file_name, all_distances):
    
    os.system('rm -f %s' % file_name)

    with open(file_name, 'a') as writer:
    
        for i in range(len(all_distances)):
            item = all_distances[i]
            writer.write(json.dumps(item) + "\n")
            if i % 100 == 0 or i == len(all_distances) - 1:
                progressBar(i + 1, len(all_distances), 'Writing Annotations to File:    ' + file_name)

In [16]:
# file_name: file name of occlusion annotation file
def write_occlusions(file_name, all_occlusions):
    
    os.system('rm -f %s' % file_name)

    with open(file_name, 'a') as writer:
        for i in range(len(all_occlusions)):
            item = all_occlusions[i]
            writer.write(json.dumps(item) + "\n")
            if i % 100 == 0 or i == len(all_occlusions) - 1:
                progressBar(i + 1, len(all_occlusions), 'Writing Annotations to File:    ' + file_name)

In [17]:
# dataset: 'train', 'valid', or 'test'
def build_and_write_annotations(dataset):
    all_distances, all_occlusions, skipped_images_ids, skipped_indices, num_skipped_images = build_annotations(dataset)
    
    file_name_diatances = 'annotation_' + dataset + '_distances' + '.odgt'
    file_name_occlusions = 'annotation_' + dataset + '_occlusions' + '.odgt'
    
    print()
    write_distances(file_name_diatances, all_distances)
    
    print()
    write_occlusions(file_name_occlusions, all_occlusions)

# Build and Write Annotations

## Training Set

In [18]:
#build_and_write_annotations('train')

## Validation Set

In [19]:
#build_and_write_annotations('valid')

## Test Set

In [20]:
#build_and_write_annotations('test')